ref: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [12]:
##### Function to apply text analysis to a column with `colname` in the data file `filename`


#----------------------------------------------------------------------------------------------------
### Input ###
        # colname: colname name of the variable
        # filename: filename of the data
        # var_explain: the threshold of the variance explained to select the top PCs in PCA analysis, 
                        # value between (0,1)
        # savename: name of the data file generated
        
### Input ###
        # a data file with each column named as `colname_pci`, saved in local directory as `savename`
#----------------------------------------------------------------------------------------------------        



def text_analysis(colname = "plot outline", filename = "feature_multi_top100.txt", var_explain = 0.9,
                  savename = "plot_outline_text_analysis.csv"):
    
    import pandas as pd
    import numpy as np
    import re
    import warnings
    warnings.filterwarnings("ignore")
    
    
    
    ### --------------- Load Data --------------- ###
    df = pd.read_csv(filename)
    df_col = df[colname]
    
    
    ## --------------- Bag-of-Words --------------- ##
    
    ## string to list
    import re
    col_words = []
    
    for i in range(len(df_col)):
    
        if type(df_col[i]) == str: 
            letters_only = re.sub("[^a-zA-Z]", " " , df_col[i]) # remove non-letter
            lower_case = letters_only.lower().split()   # Convert to lower case # Split into words
            
            # avoid downloading nltk
            # from NLTK stopwords https://pythonprogramming.net/stop-words-nltk-tutorial/
            stops = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 
                     'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 
                     'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 
                     's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 
                     'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
                     'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 
                     'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 
                     'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 
                     'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 
                     'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 
                     'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 
                     'it', 'how', 'further', 'was', 'here', 'than'} 
            meaningful_words = [w for w in lower_case if not w in stops]  # Remove stop words from "words"
            
            words = ( " ".join(meaningful_words))
    
        else: words = "NA"
       
        col_words.append(words)
        
        
    
    ## list to vector
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object
    vectorizer = CountVectorizer(analyzer = "word",   
                                 tokenizer = None,    
                                 preprocessor = None, 
                                 stop_words = None,   
                                 max_features = 10000)

    col_data = vectorizer.fit_transform(col_words)
    col_data = col_data.toarray()

    
    ## --------------- PCA --------------- ##
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components = var_explain, svd_solver = "full") # keep the first n PCs with 90% variance explained
    df_pca = pd.DataFrame(pca.fit_transform(col_data))
    
    ## ------------- Align with imdb_id and save ------------##
    
    df_new_pca = pd.concat([df["imdb_ids"], df_pca], axis = 1)

    col_names = ["imdb_ids"]
    for i in range(df_pca.shape[1]):
        i_name = colname + "_PC" + str(i)
        col_names.append(i_name)
    
    df_new_pca.columns = col_names
    df_new_pca.to_csv(savename, index = False)
    


In [ ]:
text_analysis(colname = "title", filename = "imdb_top100_data_parse.txt", var_explain = 0.6,
                  savename = "top100_title_text_analysis.csv")

In [11]:
import pandas as pd
pd.read_csv("top100_title_text_analysis.csv").head()

,imdb_ids,title_PC0,title_PC1,title_PC2,title_PC3,title_PC4,title_PC5,title_PC6,title_PC7,title_PC8,...,title_PC26,title_PC27,title_PC28,title_PC29,title_PC30,title_PC31,title_PC32,title_PC33,title_PC34,title_PC35
0,113101,-0.079327,-0.011353,-0.029817,-0.043224,-0.029065,-4.421113e-17,-0.045250,1.423015e-18,-1.786223e-17,...,1.760413e-15,0.096727,-6.794354e-15,-0.129179,-0.039168,-0.049688,-4.599880e-18,-7.283144e-18,6.064175e-17,2.759928e-17
1,425473,-0.088179,-0.015068,-0.040108,-0.059759,-0.042426,2.918354e-16,-0.068966,-1.867128e-16,-2.871110e-16,...,1.454471e+00,-0.343248,1.165585e-14,0.136044,0.037079,0.017544,2.925731e-16,-3.216521e-16,-6.671028e-16,8.106811e-16
2,76759,0.656727,-0.024725,-0.016591,-0.014724,-0.008640,-6.501939e-15,-0.013441,-8.728284e-17,-3.070047e-16,...,-5.573449e-16,0.026175,-7.075205e-16,-0.017686,-0.063346,-0.008722,-6.192331e-16,-1.851807e-16,3.096729e-16,-5.711407e-16
3,266543,-0.079327,-0.011353,-0.029817,-0.043224,-0.029065,7.187369e-16,-0.045250,-1.863760e-16,-6.857209e-16,...,1.152621e-15,0.096727,-5.822481e-15,-0.129179,-0.039168,-0.049688,-1.168951e-01,1.538504e-01,-3.960839e-03,-1.546434e-02
4,411267,-0.080060,-0.011935,-0.031466,-0.045981,-0.031436,1.504055e-15,-0.049616,-3.092303e-16,1.947978e-17,...,1.603433e-15,0.345515,6.015010e-01,0.447650,0.102126,0.024368,6.012173e-16,-1.055284e-16,1.796691e-16,-1.591275e-17
